### 과제 url

url => https://www.musinsa.com/app/codimap/lists?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=comment_cnt&page=1

과제: 무신사 코디맵 페이지(많은 댓글 수 기준 정렬)에 나와있는 60개의 코디맵에 대하여 각각 ['codimap_category', 'codimap_title', 'codimap_date', 'views', 'comment_numbers'] 정보를 가져와서 csv 형태로 만든 다음, 해당 코드(.ipynb)와 csv 파일을 본인 github 과제 repository에 제출

Challenge:
1. 각 코디맵마다 클릭한 뒤 다시 이전페이지로 돌아오는 코드를 반복문에 추가하여 ['codimap_explain', 'codimap_hashtag', 'codimap_imgurl'] 정보도 가져오기
2.  page url을 format 함수를 이용해 수정한 뒤 반복문을 통해 1페이지부터 5페이지까지의 코디맵 정보 가져오기

# Challenge는 Optional Task로 필수적으로 해야하는 것은 아닙니다. 좀 더 심화된 크롤링을 원할경우 시도해보면 좋을 것 같아요!

In [ ]:
!pip3 install requests
!pip3 install bs4
!pip3 install selenium

In [1]:
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import numpy as np
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
print(selenium.__version__)

4.10.0


In [2]:
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

In [3]:
# 무신사 창 불러오기
url = 'https://www.musinsa.com/app/codimap/lists?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=comment_cnt&page=1'
driver.get(url)
driver.implicitly_wait(2)
try:
    driver.maximize_window()
except:
    pass

In [4]:
interval = 1
prev_height = driver.execute_script("return document.body.scrollHeight")
while True:
        # 스크롤을 가장 아래로 내림
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")

        # 페이지 로딩 대기
        time.sleep(interval)

        # 현재 문서 높이를 가져와서 저장
        curr_height = driver.execute_script("return document.body.scrollHeight")
        if curr_height == prev_height:
            break

        prev_height = curr_height

In [7]:
columns = ['codimap_category', 'codimap_title', 'codimap_date', 'views', 'comment_numbers']
values = []
soup = BeautifulSoup(driver.page_source, 'lxml')
data_rows = soup.find_all('li', attrs={'class':'style-list-item'})

for i, row in enumerate(data_rows):
    print('{}번째 코디맵 크롤링'.format(i+1))
    codimap = []
    
    category = row.find('span', attrs={'class':'style-list-information__text'})
    if category:
        category = category.get_text().strip()
        codimap.append(category)
    else:
        codimap.append('Something is wrong')
        print(f'{i+1}번째 카테고리 가져올때 문제발생')
        continue

    title = row.find('strong', attrs={'class':'style-list-information__title'})
    if title:
        title = title.get_text().strip()
        codimap.append(title)
    else:
        codimap.append('Something is wrong')
        print(f'{i+1}번째 타이틀 가져올때 문제발생')
        continue
    
    information = row.find_all('span', attrs={'class':'post-information__text'})
    print(information)
    print(information[0], information[1], information[2])
              
    date = information[0]                 
    if date:
        date = date.get_text().replace('""','').strip()
        codimap.append(date)
    else:
        codimap.append('Something is wrong')
        print(f'{i+1}번째 날짜 가져올때 문제발생')
        continue
    
    views = information[1]                 
    if views:
        views = views.get_text().replace('""','').strip()
        views = views[3:]        # 또는 views = re.sub(r'[^0-9]', '', views) re 사용
        codimap.append(views)
    else:
        codimap.append('Something is wrong')
        print(f'{i+1}번째 조회수 가져올때 문제발생')
        continue
    
    comments = information[2]           
    if comments:
        comments = comments.get_text().replace('""','').strip()
        comments = comments[3:]
        codimap.append(comments)
    else:
        codimap.append('Something is wrong')
        print(f'{i+1}번째 댓글수 가져올때 문제발생')
        continue
    
    values.append(codimap)
    print(codimap)
    print('---------------------------------------------------')


1번째 코디맵 크롤링
[<span class="post-information__text">21.07.02</span>, <span class="post-information__text">조회 16,026</span>, <span class="post-information__text">댓글 49</span>]
<span class="post-information__text">21.07.02</span> <span class="post-information__text">조회 16,026</span> <span class="post-information__text">댓글 49</span>
['캐주얼', '톤온톤 코디', '21.07.02', '16,026', '49']
---------------------------------------------------
2번째 코디맵 크롤링
[<span class="post-information__text">21.02.23</span>, <span class="post-information__text">조회 34,663</span>, <span class="post-information__text">댓글 46</span>]
<span class="post-information__text">21.02.23</span> <span class="post-information__text">조회 34,663</span> <span class="post-information__text">댓글 46</span>
['캐주얼', '블랙 매니아', '21.02.23', '34,663', '46']
---------------------------------------------------
3번째 코디맵 크롤링
[<span class="post-information__text">22.03.22</span>, <span class="post-information__text">조회 12,192</span>, <span class="post-info

In [9]:
df = pd.DataFrame(values, columns=columns)
display(df)

,codimap_category,codimap_title,codimap_date,views,comment_numbers
0,캐주얼,톤온톤 코디,21.07.02,"16,026",49
1,캐주얼,블랙 매니아,21.02.23,"34,663",46
2,캐주얼,오늘의 코디,22.03.22,"12,192",38
3,댄디,추천 댄디 룩,21.09.15,"20,178",27
4,캐주얼,릴랙스 룩,21.02.23,"23,529",22
5,스트릿,힙하다 힙해,21.02.03,"35,279",21
6,캐주얼,심플한 코디,21.07.30,"39,565",18
7,댄디,데이트 코디,21.12.24,"10,303",18
8,댄디,훈훈 남친 룩,21.03.24,"19,337",16
9,캐주얼,지금 입기 좋아,21.08.31,"20,206",16


In [10]:
df.to_csv('김채연_크롤링과제.csv', encoding = 'utf-8-sig')